Import

In [ ]:
import pandas as pd
import json
import tensorflow as tf
import matplotlib.pyplot as plt
import mlflow
from mlflow.models import infer_signature

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlruns.db")

In [ ]:
# ========== Dataset from Drive ==========
# from google.colab import drive
# drive.mount('/data/data')

Classes

In [ ]:
import os
for dir, sub_dirs, _ in os.walk("./data"):
    class_names = sub_dirs
    break

json.dump({"classes": class_names}, open("classes.json", "w"))
n_classes = len(class_names)
class_names

In [ ]:
# ========== Dataset loading ==============

directory = "./data"
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    class_names=class_names,
    color_mode='rgb',
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=120,
    validation_split=0.30,
    subset="training",
    verbose=True)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    class_names=class_names,
    color_mode='rgb',
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=120,
    validation_split=0.30,
    subset="validation",
    verbose=True)

# ========== Data Preprocessing ==============
def image_rescaling(img, y):
    img = img * (1./255)
    return img, y

def onehot_encode(img, y):
    y = tf.one_hot(y, n_classes)
    return img, y


train_dataset = train_dataset.map(image_rescaling)
train_dataset = train_dataset.map(onehot_encode)

validation_dataset = validation_dataset.map(image_rescaling)
validation_dataset = validation_dataset.map(onehot_encode)

# =========== Prefetchig to efficiently load the dataset in memory =========
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE) 


In [ ]:
for img, y in train_dataset:
    print("img:", plt.imshow(img[0].numpy()))
    print("target:",y[0])
    break
plt.show()

#### Modelling

In [ ]:
def get_model():
    # Input Layer
    image_input = tf.keras.Input(shape=(128, 128, 3), name='image_input')
    # Convolution Layers
    x = tf.keras.layers.Conv2D(16, 3, activation="relu")(image_input)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Conv2D(32, 3, activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Conv2D(64, 3, activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = tf.keras.layers.Conv2D(128, 3, activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Conv2D(128, 3, activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    ## Fully Connected Layers
    features_output = tf.keras.layers.Flatten()(x)
    x =  tf.keras.layers.Dense(128, activation="relu")(features_output)
    x =  tf.keras.layers.Dense(64, activation="relu")(x)
    output =  tf.keras.layers.Dense(n_classes, name="output")(x)

    # Model Definition
    model = tf.keras.Model(image_input, output, name="mushroom-model")
    model.summary()
    return model

In [ ]:
model = get_model()
# ====== Model Compile ======
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.Accuracy()],
              )

# optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3)
# sparse_categorical_loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metrics= tf.keras.metrics.AUC()


#### Training

In [ ]:
# for x, y in train_dataset:
#     with tf.GradientTape() as tape:
#         output_logits = model(x, training=True)
#         loss = sparse_categorical_loss()

model.fit(train_dataset, epochs=10)
model.save("mushroom_classifier.keras")

In [ ]:
# model evaluation
output_evaluation = model.evaluate(validation_dataset)
output_evaluation

Mlflow-Tracking

In [ ]:
experiment_name = "mushroom_image_classifier"
try:
    eid = mlflow.create_experiment(name=experiment_name)
except:
    experiment_obj = mlflow.get_experiment_by_name(name=experiment_name)
    eid = experiment_obj.experiment_id
    
with mlflow.start_run(experiment_id=eid) as run:
    mlflow.tensorflow.log_model(model, "mashroom")
    run_id = run.info.run_id
    print("run_id", run_id)
    print("exp_id", eid)